In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/down_sample.csv')[['Subreddit', 'unst_simp_lem']]

In [4]:
df.head()

,Subreddit,unst_simp_lem
0,0,i hate feel get interested someone recently go...
1,0,how make friend someone suggest something make...
2,0,so today birthday 9 11 hoping good day well iv...
3,0,having hard time need someone say this dont wa...
4,0,anti depressant inducing trip like state exper...


In [5]:
X = df.drop(columns='Subreddit')
y = df['Subreddit']

X_train, X_test, y_train, y_test=train_test_split(X, y, stratify = y, random_state=42)

In [6]:
cvec = CountVectorizer()
cvec.fit(X_train['unst_simp_lem'])
X_train_cv = pd.DataFrame(cvec.transform(X_train['unst_simp_lem']).toarray(), columns = cvec.get_feature_names())
X_test_cv = pd.DataFrame(cvec.transform(X_test['unst_simp_lem']).toarray(), columns = cvec.get_feature_names())

tvec = TfidfVectorizer()
tvec.fit(X_train['unst_simp_lem'])
X_train_tv = pd.DataFrame(tvec.transform(X_train['unst_simp_lem']).todense(), columns = tvec.get_feature_names())
X_test_tv = pd.DataFrame(tvec.transform(X_test['unst_simp_lem']).todense(), columns = tvec.get_feature_names())

In [7]:
sc = StandardScaler()
sc.fit(X_train_tv)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
X_train_tv_sc = sc.transform(X_train_tv)

In [9]:
X_test_tv_sc = sc.transform(X_test_tv)

In [11]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_tv_sc.shape[1]))
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()], optimizer='adam')

In [12]:
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, 
                           verbose=1, mode='auto')

In [13]:
history = model.fit(X_train_tv_sc, y_train, 
                    validation_data=(X_test_tv_sc, y_test),
                    epochs=50, 
                    batch_size=64,
                    verbose=1, 
                    callbacks=[early_stop])

Epoch 1/50
352/352 [==============================] - 6s 9ms/step - loss: 0.8465 - accuracy: 0.6305 - precision: 0.6271 - val_loss: 0.5594 - val_accuracy: 0.7128 - val_precision: 0.7796
Epoch 2/50
352/352 [==============================] - 2s 7ms/step - loss: 0.4891 - accuracy: 0.7945 - precision: 0.7910 - val_loss: 0.5695 - val_accuracy: 0.7175 - val_precision: 0.7061
Epoch 3/50
352/352 [==============================] - 2s 7ms/step - loss: 0.2804 - accuracy: 0.8908 - precision: 0.8831 - val_loss: 0.7227 - val_accuracy: 0.7107 - val_precision: 0.7199
Epoch 4/50
352/352 [==============================] - 2s 7ms/step - loss: 0.1542 - accuracy: 0.9428 - precision: 0.9360 - val_loss: 1.1744 - val_accuracy: 0.6995 - val_precision: 0.7021
Epoch 5/50
352/352 [==============================] - 2s 7ms/step - loss: 0.0829 - accuracy: 0.9698 - precision: 0.9689 - val_loss: 1.6231 - val_accuracy: 0.6984 - val_precision: 0.7169
Epoch 6/50
352/352 [==============================] - 2s 7ms/step - lo